<div class="alert alert-block alert-success">
    
# FIT5196 Task 1 in Assessment 1
#### Student Name: Emmanuelle Christin; Phakhanan Rataphaibul
#### Student ID: 32941943; 33654735

Date: 16 March 2024


Environment: Python 3.10.12

Libraries used:
* re (for regular expression, installed and imported)
* pandas (for data manipulation)
    
</div>

<div class="alert alert-block alert-danger">
    
## Table of Contents

</div>    

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Patent Files](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Defining Regular Expressions](#Reg_Exp) <br>
$\;\;\;\;$[4.2. Reading Files](#Read) <br>
$\;\;\;\;$[4.3. Whatever else](#latin) <br>
[5. Writing to CSV/JSON File](#write) <br>
$\;\;\;\;$[5.1. Verification - using the sample files](#test_xml) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

-------------------------------------

<div class="alert alert-block alert-warning">

## 1.  Introduction  <a class="anchor" name="Intro"></a>
    
</div>

This assessment regards extracting data from semi-structured text files related to a subset of records on trademark assignments. The dataset is a `.txt` file stored in XML format, each representing a unique trademark assignment. In particular, each text file is recorded with a set of attributes such as reel no, frame no, correspondent party, assignors, assignee, and properties. The primary objective of this assessment is to extract, parse, and transform these semi-structured text files into JSON format which is a more structured and analyzable format.

Key tasks in this assessment involve:

1. Examining raw data from the .txt files to understand the structure and identify patterns for different data elements.
2. Parsing the data using regular expressions (re library) to extract specific patterns and information from the semi-structured text files across all fields.
3. Further processing the extracted data from step 2 to refine the required fields according to the specified requirements and save the data into a proper format.
4. Writing the processed data to a JSON output file.

-------------------------------------

<div class="alert alert-block alert-warning">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>
 </div>

The packages to be used in this assessment are imported in the following. They are used to fulfill the following tasks:

* **re:** is imported to define and use regular expressions in Python. In this assessment, regular expressions (regex) are utilized for defining patterns and extracting specific data from XML files. Regular expressions are used to match and extract various elements such as assignment details, correspondent information, assignor and assignee details, and properties information from the structured XML data.
* **json:**  is imported to handle JSON (JavaScript Object Notation) data in Python. JSON is a lightweight and human-readable data interchange format commonly used for transmitting and storing data objects. In the context of this assessment, the json library is utilized to work with JSON data. Specifically, it is used to write JSON data to a file in a structured and formatted manner.

In [1]:
import re
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


-------------------------------------

<div class="alert alert-block alert-warning">

## 3.  Examining Raw Data <a class="anchor" name="examine"></a>

 </div>

First of all, understanding the structure and content of the raw data is necessary for extracting meaningful information. Having examined the file content, the following observations were made:

- Most data entries do not have date-acknowledged field.
- A single correspondent may have up to four addresses listed.
- There can be many assignors, assignees, and properties.
- A single assignor/assignee can have multiple addresses.
- The field 'nationality' is sometimes specified with a state or city name instead of a country name.
- Similarly, the field 'country-name' is also specified with a state or city name instead of the actual country name
- Some organization names include titles such as 'Mr.' within the name.
- When the 'legal-entity-text' field of an assignor/assignee is set to 'INDIVIDUAL', the 'person-or-organization-name' field will contain a person's name. Conversely, when it is set to 'CORPORATION', the 'person-or-organization-name' field will contain an organization's name
- Some fields contain special character such as French name in 'legal-entity-text'.


-------------------------------------

<div class="alert alert-block alert-warning">

## 4.  Loading and Parsing Files <a class="anchor" name="load"></a>

</div>

This section consists of three main processes. We start by defining appropriate regular expressions to extract necessary fields from the XML content. We then parse the file using the defined regular expressions and functions, finally and transform the parsed data into JSON format.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.1. Defining Regular Expressions <a class="anchor" name="Reg_Exp"></a>

In this step, we define a set of regular expressions for extracting all necessary fields from the XML content. These patterns are designed to match various elements within the XML structure, including details regarding assignment entries such as reel numbers, frame numbers, and last updated dates, and details concerning correspondents, assignors, assignees, and property-related information.

In [3]:
# Regular expression patterns to extract data from XML content
assg_entry_pattern = r'<assignment-entry>(.*?)</assignment-entry>' # Pattern to find assignment entries
assg_pattern = r'<assignment>(.*?)</assignment>' # Pattern to match assignment data
reel_no_pattern = r'<reel-no>(\d+)</reel-no>' # Pattern to match reel number
frame_no_pattern = r'<frame-no>(.*?)</frame-no>' # Pattern to match frame number
update_date_pattern = r'<last-update-date>(.*?)</last-update-date>' # Pattern to match last update date
purge_indicator_pattern = r'<purge-indicator>(.*?)</purge-indicator>' # Pattern to match purge indicator
date_recorded_pattern = r'<date-recorded>(.*?)</date-recorded>' # Pattern to match date recorded
page_count_pattern = r'<page-count>(.*?)</page-count>' # Pattern to match page count
correspondent_person_pattern = r'<person-or-organization-name>(.*?)</person-or-organization-name>' # Pattern to match correspondent person/organization name
correspondent_address1_pattern = r'<address-1>(.*?)</address-1>' # Pattern to match correspondent address 1
correspondent_address2_pattern = r'<address-2>(.*?)</address-2>' # Pattern to match correspondent address 2
correspondent_address3_pattern = r'<address-3>(.*?)</address-3>' # Pattern to match correspondent address 3
correspondent_address4_pattern = r'<address-4>(.*?)</address-4>' # Pattern to match correspondent address 4
correspondent_pattern = r'<correspondent>(.*?)</correspondent>' # Pattern to match correspondent
conveyance_text_pattern = r'<conveyance-text>(.*?)</conveyance-text>' # Pattern to match conveyance text
assignor_person_pattern = r'<person-or-organization-name>(.*?)</person-or-organization-name>' # Pattern to match assignor person/organization name
assignor_state_pattern = r'<state>(.*?)</state>' # Pattern to match assignor state
assignor_execution_date_pattern = r'<execution-date>(.*?)</execution-date>' # Pattern to match assignor execution date
assignor_legal_entity_text_pattern = r'<legal-entity-text>(.*?)</legal-entity-text>' # Pattern to match assignor legal entity text
assignor_date_acknowledged = r'<date-acknowledged>(.*?)</date-acknowledged>' # Pattern to match assignor date acknowledged
assignor_nationality_pattern = r'<nationality>(.*?)</nationality>'  # Pattern to match assignor nationality
assignor_country_pattern = r'<country-name>(.*?)</country-name>' # Pattern to match assignee country name
assignee_person_pattern = r'<person-or-organization-name>(.*?)</person-or-organization-name>' # Pattern to match assignee person/organization name
assignee_address1_pattern = r'<address-1>(.*?)</address-1>' # Pattern to match assignee address 1
assignee_address2_pattern = r'<address-2>(.*?)</address-2>' # Pattern to match assignee address 2
assignee_address3_pattern = r'<address-3>(.*?)</address-3>' # Pattern to match assignee address 3
assignee_address4_pattern = r'<address-4>(.*?)</address-4>' # Pattern to match assignee address 4
assignee_city_pattern = r'<city>(.*?)</city>' # Pattern to match assignee city
assignee_country_pattern = r'<country-name>(.*?)</country-name>' # Pattern to match assignee country name
assignee_state_pattern = r'<state>(.*?)</state>' # Pattern to match assignee state
assignee_postcode_pattern = r'<postcode>(.*?)</postcode>' # Pattern to match assignee postal code
assignee_legal_entity_text_pattern = r'<legal-entity-text>(.*?)</legal-entity-text>' # Pattern to match assignee legal entity text
assignee_nationality_pattern = r'<nationality>(.*?)</nationality>' # Pattern to match assignee nationality
property_serial_no_pattern = r'<serial-no>(.*?)</serial-no>' # Pattern to match property serial number
property_registration_no_pattern = r'<registration-no>(.*?)</registration-no>' # Pattern to match property registration number
properties_pattern = r'<properties>(.*?)</properties>' # Pattern to match properties section
property_pattern = r'<property>(.*?)</property>' # Pattern to match property entry
assignors_pattern = r'<assignors>(.*?)</assignors>' # Pattern to match assignors section
assignor_pattern = r'<assignor>(.*?)</assignor>' # Pattern to match assignor entry
assignees_pattern = r'<assignees>(.*?)</assignees>' # Pattern to match assignees section
assignee_pattern = r'<assignee>(.*?)</assignee>' # Pattern to match assignee entry

These patterns will be used in the next step when reading the files.

-------------------------------------

<div class="alert alert-block alert-info">
    
### 4.2. Parsing and Reading Files <a class="anchor" name="Read"></a>

In this step, we defined three important functions to facilitate the parsing and reading of XML files containing trademark assignments records.

1. The 'unescape_and_convert' function is tasked with decoding Unicode escape sequences and replacing XML special characters such as '&', '<', '>', '"', and ''' with their respective symbols. This preprocessing is necessary to ensure that the XML data is properly formatted and ready for parsing.

2. The 'safe_search' function is designed to safely execute regular expression searches for some fields. It returns the matched group if found or None if the match is unsuccessful, thus preventing errors during data extraction when expected content is missing in the XML structure.

3. The 'parse_xml_data' function is the core parsing function responsible for extracting relevant information from the XML data. It starts by unescaping XML special characters using the previously defined function. Then, it iterates through the assignment entries in the XML data using regular expressions defined in step 4.1 to extract specific fields in the XML content.

For each assignment entry, data are parsed and organized into structured dictionaries and lists, facilitating easy access and manipulation. Details regarding assignors, assignees, and properties are extracted separately and integrated into each assignment's dictionary. After completing the parsing process, the function outputs a list of dictionaries, each representing a comprehensive assignment entry with all associated details. This parsed data is subsequently read from a specified XML file, processed through the 'parse_xml_data' function, and the first ten entries are printed to validate that the data has been parsed and stored correctly.

-------------------------------------

In [4]:
# Function to unescape XML special characters
def unescape_and_convert(text):

    """Convert Unicode escape sequences and unescape XML special characters in the string."""
    # Check if the text is in bytes, decode it to utf-8
    if isinstance(text, bytes):
        text = text.decode('utf-8')

    # Replace XML special characters with their appropriate symbols
    text = text.replace('&amp;', '&')
    text = text.replace('&lt;', '<')
    text = text.replace('&gt;', '>')
    text = text.replace('&quot;', '"')
    text = text.replace('&apos;', "'")

    return text

def safe_search(pattern, text):
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None

# Function to parse XML data
def parse_xml_data(xml_data):
  # Unescape XML special characters
  xml_data = unescape_and_convert(xml_data)

  # Initialize an empty list to store the parsed data
  parsed_data = []

  # Find assignment entries using regular expressions
  assignment_entries = re.finditer(assg_entry_pattern, xml_data, re.DOTALL)

  # Iterate over assignment entries
  for entry in assignment_entries:
    assignment_data = entry.group(1)
    # Initialize an empty dictionary for each assignment
    assignment_dict = {}

    # Extract assignment data using regular expressions
    assignment_dict['reel-no'] = re.search(reel_no_pattern, assignment_data).group(1) if re.search(reel_no_pattern, assignment_data) else None
    assignment_dict['frame-no'] = re.search(frame_no_pattern, assignment_data).group(1) if re.search(frame_no_pattern, assignment_data) else None
    assignment_dict['last-update-date'] = re.search(update_date_pattern, assignment_data).group(1) if re.search(update_date_pattern, assignment_data) else None
    assignment_dict['purge-indicator'] = re.search(purge_indicator_pattern, assignment_data).group(1) if re.search(purge_indicator_pattern, assignment_data) else None
    assignment_dict['date-recorded'] = re.search(date_recorded_pattern, assignment_data).group(1) if re.search(date_recorded_pattern, assignment_data) else None
    assignment_dict['page-count'] = re.search(page_count_pattern, assignment_data).group(1) if re.search(page_count_pattern, assignment_data) else None

    # Initialize an empty dictionary for correspondent information
    correspondent_dict = {}

    # Extract correspondent block
    correspondent_blocks = re.findall(correspondent_pattern, assignment_data, re.DOTALL)

    for block in correspondent_blocks:

        # Search for the person or organization name within the block
        correspondent_name_match = re.search(correspondent_person_pattern, block, re.DOTALL)

        if correspondent_name_match:
            correspondent_name = correspondent_name_match.group(1).strip() if correspondent_name_match.group(1).strip() else None
        else:
            correspondent_name = None

        correspondent_dict['person-or-organization-name'] = correspondent_name

    assignment_dict['correspondent'] = correspondent_dict

    assignment_dict['conveyance-text'] = re.search(conveyance_text_pattern, assignment_data).group(1) if re.search(conveyance_text_pattern, assignment_data) else None

    assignors_match = re.finditer(assignor_pattern, assignment_data, re.DOTALL)
    assignees_match = re.finditer(assignee_pattern, assignment_data, re.DOTALL)
    properties_match = re.finditer(property_pattern, assignment_data, re.DOTALL)

    # Extract assignors using regular expressions
    assignors_list = []
    for assignor_match in assignors_match:
      assignor_data = assignor_match.group(1)
      assignors_list.append({
        'person-or-organization-name': safe_search(assignor_person_pattern, assignor_data),
        'execution-date': safe_search(assignor_execution_date_pattern, assignor_data),
        'legal-entity-text':safe_search(assignor_legal_entity_text_pattern, assignor_data),
        'date-acknowledged':safe_search(assignor_date_acknowledged, assignor_data),
        'nationality':safe_search(assignor_nationality_pattern, assignor_data),
        'state':safe_search(assignor_state_pattern, assignor_data),
        'country-name':safe_search(assignor_country_pattern, assignor_data)

      })
      if assignors_list:
        assignment_dict['assignors'] = assignors_list
    assignees_list = []
    for assignee_match in assignees_match:
      assignee_data = assignee_match.group(1)
      assignees_list.append({
        'person-or-organization-name': safe_search(assignee_person_pattern, assignee_data),
        'address-1':safe_search(assignee_address1_pattern, assignee_data),
        'address-2':safe_search(assignee_address2_pattern, assignee_data),
        'address-4':safe_search(assignee_address4_pattern, assignee_data),
        'country-name':safe_search(assignee_country_pattern, assignee_data),
        'city':safe_search(assignee_city_pattern, assignee_data),
        'state':safe_search(assignee_state_pattern, assignee_data),
        'postcode':safe_search(assignee_postcode_pattern, assignee_data),
        'legal-entity-text':safe_search(assignee_legal_entity_text_pattern, assignee_data),
        'nationality':safe_search(assignee_nationality_pattern, assignee_data)
      })
      assignment_dict['assignees'] = assignees_list

    # Extract properties data using regular expressions
      properties_list = []
      for property_match in properties_match:
        property_data = property_match.group(1)
        property_dict = {
          'serial-no': safe_search(property_serial_no_pattern, property_data),
          'registration-no': safe_search(property_registration_no_pattern, property_data)
        }
        properties_list.append(property_dict)
        assignment_dict['properties'] = properties_list

    # Create an assignment entry with assignment data
    assignment = {'assignment': assignment_dict}
    parsed_data.append(assignment)

  return parsed_data

# File path to the XML data file
data_file_path = '/content/drive/Shareddrives/FIT5196_S1_2024/A1/Students data/Task 1/Group005.txt'

# Read the XML data from the file
with open(data_file_path, "r", encoding = "utf-8") as file:
    xml_data = file.read()

# Parse the XML data to extract assignment information
parsed_data = parse_xml_data(xml_data)

# Print the first ten elements
print(parsed_data[:10])

[{'assignment': {'reel-no': '6409', 'frame-no': '0912', 'last-update-date': '20180813', 'purge-indicator': 'N', 'date-recorded': '20180809', 'page-count': '5', 'correspondent': {'person-or-organization-name': 'WORLD HOLDINGS INC.'}, 'conveyance-text': 'ASSIGNS THE ENTIRE INTEREST', 'assignors': [{'person-or-organization-name': 'ISODIOL INTERNATIONAL INC', 'execution-date': '20180702', 'legal-entity-text': 'CORPORATION', 'date-acknowledged': None, 'nationality': 'CANADA', 'state': None, 'country-name': None}], 'assignees': [{'person-or-organization-name': 'WORLD HOLDINGS, INC.', 'address-1': '1712 PIONEER AVENUE SUITE 500', 'address-2': None, 'address-4': None, 'country-name': None, 'city': 'CHEYENNE', 'state': 'WYOMING', 'postcode': '82001', 'legal-entity-text': 'CORPORATION', 'nationality': 'WYOMING'}], 'properties': [{'serial-no': '77962789', 'registration-no': '4354279'}]}}, {'assignment': {'reel-no': '3792', 'frame-no': '0180', 'last-update-date': '20080609', 'purge-indicator': 'N'

<div class="alert alert-block alert-info">
    
### 4.3. Transform to JSON Format <a class="anchor" name="latin"></a>

In this step, once again, we defined two functions for handling specific tasks as follows:

1. The 'remove_title_from_person_name' function is designed to cleanse personal names by removing the titles. It operates by checking whether a particular assignor/assignee is labeled as an individual or an organization; if NOT labeled as an individual, it bypasses the cleansing. Otherwise, it utilizes a predefined list of titles, which it removes from the name to ensure consistency and professionalism in data representation.

2. The 'get_country' function extracts the country information from an entry's dictionary. It prioritizes the 'country-name', 'state', and 'nationality' keys to ascertain the most accurate country assignment. This function includes normalization mappings to handle variations such as 'UNITED STATES' to 'USA' and returns 'NA' for cases where neither 'country-name', 'state', nor 'nationality' is provided.

Consequently, we further process the parsed XML data from the previous steps to transform it into a structured JSON format. The process involves using the defined functions to handle specific tasks such as removing titles from person names and extracting country information from assignor/assignee data. Upon parsing the XML data and extracting relevant assignment details, we embark on transforming this data into a structured JSON format. Each entry is transformed into a dictionary format, with fields organized systematically. Assignment entry details, assignors, assignees, and properties information are appropriately structured within each entry to be further processed in the next step. Lastly, we printed the first five elements of the transformed data to verify the transformation results and inspect the JSON structure.

In [5]:
def remove_title_from_person_name(name, legal_entity_text):

    # Define a list for titles to be removed
    title_list = ["Mr", "Mrs", "Miss", "Ms", "Mx", "Sir", "Dame", "Dr", "Cllr", "Lady", "Lord"]
    title_list_with_period = [title + '.' for title in title_list]
    title_list = [title.upper() for title in title_list]

    title_list_combined = title_list + title_list_with_period
    title_list_combined = [title.upper() for title in title_list_combined]

    # Check if the legal entity is not an individual
    if (legal_entity_text or 'NA').upper() != "INDIVIDUAL":
        return name  # Return the name as is if not an individual

    # Split the name into parts, remove any dots, and convert each to uppercase for comparison
    name_parts = name.split()
    cleaned_name_parts = []
    for part in name_parts:
        # Check if the part is a title and should be removed
        if part.upper() not in title_list_combined:
            cleaned_name_parts.append(part)

    # Reconstruct the name without the titles
    cleaned_name = ' '.join(cleaned_name_parts)
    # Properly handle commas and trailing spaces
    cleaned_name = cleaned_name.strip(', ')

    return cleaned_name

# Function to get the country from an assignor/assignee
def get_country(entry):
    country_normalization = {
        'UNITED STATES': 'USA',
        'UNITED KINGDOM': 'UK',
        'ENGLAND':'UK',
        'USA': 'USA'
    }

    state_to_country = {

    # USA states
    'ALABAMA': 'USA', 'ALASKA': 'USA', 'ARIZONA': 'USA', 'ARKANSAS': 'USA', 'CALIFORNIA': 'USA',
    'COLORADO': 'USA', 'CONNECTICUT': 'USA', 'DELAWARE': 'USA', 'FLORIDA': 'USA', 'GEORGIA': 'USA',
    'HAWAII': 'USA', 'IDAHO': 'USA', 'ILLINOIS': 'USA', 'INDIANA': 'USA', 'IOWA': 'USA', 'KANSAS': 'USA',
    'KENTUCKY': 'USA', 'LOUISIANA': 'USA', 'MAINE': 'USA', 'MARYLAND': 'USA', 'MASSACHUSETTS': 'USA',
    'MICHIGAN': 'USA', 'MINNESOTA': 'USA', 'MISSISSIPPI': 'USA', 'MISSOURI': 'USA', 'MONTANA': 'USA',
    'NEBRASKA': 'USA', 'NEVADA': 'USA', 'NEW HAMPSHIRE': 'USA', 'NEW JERSEY': 'USA', 'NEW MEXICO': 'USA',
    'NEW YORK': 'USA', 'NORTH CAROLINA': 'USA', 'NORTH DAKOTA': 'USA', 'OHIO': 'USA', 'OKLAHOMA': 'USA',
    'OREGON': 'USA', 'PENNSYLVANIA': 'USA', 'RHODE ISLAND': 'USA', 'SOUTH CAROLINA': 'USA',
    'SOUTH DAKOTA': 'USA', 'TENNESSEE': 'USA', 'TEXAS': 'USA', 'UTAH': 'USA', 'VERMONT': 'USA',
    'VIRGINIA': 'USA', 'WASHINGTON': 'USA','DISTRICT OF COLUMBIA':'USA', 'WEST VIRGINIA': 'USA', 'WISCONSIN': 'USA', 'WYOMING': 'USA',

    # UK regions
    'ENGLAND': 'UK', 'SCOTLAND': 'UK', 'WALES': 'UK', 'NORTHERN IRELAND': 'UK',

    # Canadian provinces
    'ONTARIO': 'CANADA', 'QUEBEC': 'CANADA', 'BRITISH COLUMBIA': 'CANADA', 'ALBERTA': 'CANADA',
    'MANITOBA': 'CANADA', 'SASKATCHEWAN': 'CANADA', 'NOVA SCOTIA': 'CANADA', 'NEW BRUNSWICK': 'CANADA',
    'NEWFOUNDLAND AND LABRADOR': 'CANADA', 'PRINCE EDWARD ISLAND': 'CANADA', 'NORTHWEST TERRITORIES': 'CANADA',
    'YUKON': 'CANADA', 'NUNAVUT': 'CANADA'
}

    # Check for direct country name
    country = entry.get('country-name')
    if country is not None:
      country = country.upper()  # Convert to uppercase to handle case sensitivity
      if country in country_normalization:
        return country_normalization[country]  # Return the normalized country name
      else:
        return country  # Return the country name as is


    # Check for state as next option using the mapping
    state = entry.get('state')
    if state and state.upper() in state_to_country:
        return state_to_country[state.upper()]

    # Finally, check for nationality if state doesn't give a conclusive result
    nationality = entry.get('nationality')
    if nationality and nationality.upper() in state_to_country:
      return state_to_country[nationality.upper()]
    elif nationality =='NOT PROVIDED':
      nationality='NA'
      return nationality
    elif nationality in country_normalization:
      return country_normalization[nationality]
    elif nationality!=None:
      return nationality

    # Return 'NA' if no country information is found
    return 'NA'

# Initialize an empty dictionary for transformed data
transformed_data = {}

# Iterate over the parsed data to transform and populate the transformed_data dictionary
for entry in parsed_data:
  # Initialize an empty dictionary for each entry transformation
  transformed_entry = {}

  # Concatenate reel-no and frame-no for rf_id
  rf_id = f"{entry['assignment']['reel-no']}{entry['assignment']['frame-no']}"

  # Transform and populate fields for the current entry
  last_update_date = entry['assignment'].get('last-update-date')
  transformed_entry['last-update-date'] = f"{last_update_date[:4]}-{last_update_date[4:6]}-{last_update_date[6:]}" # Format last-update-date
  transformed_entry['conveyance-text'] = entry['assignment'].get('conveyance-text') or 'NA'
  transformed_entry['correspondent-party'] = entry['assignment']['correspondent'].get('person-or-organization-name') or 'NA'

  # Transform assignors-info
  assignors_info = []
  # Iterate over the assignors list
  for assignor in entry['assignment']['assignors']:
    date_acknowledged = assignor.get('date-acknowledged', 'NA')
    execution_date = assignor.get('execution-date', 'NA')
    assignor_info = {
      'party-name': remove_title_from_person_name(assignor['person-or-organization-name'],assignor.get('legal-entity-text', 'NA')),
      'date-acknowledged' : date_acknowledged if date_acknowledged == 'NA' else f"{date_acknowledged[:4]}-{date_acknowledged[4:6]}-{date_acknowledged[6:]}" if date_acknowledged else 'NA',
      'execution-date': execution_date if execution_date == 'NA' else f"{execution_date[:4]}-{execution_date[4:6]}-{execution_date[6:]}" if execution_date else 'NA',
      'country': get_country(assignor), # Get country for assignor
      'legal-entity-text': assignor.get('legal-entity-text') or 'NA'
    }
    # Append assignor to assignors_info list
    assignors_info.append(assignor_info)

  # Transform assignees-info
  assignees_info = []
  # Iterate over the assignors list
  for assignee in entry['assignment']['assignees']:
    assignee_info = {
      'party-name': remove_title_from_person_name(assignee['person-or-organization-name'],assignee.get('legal-entity-text', 'NA')), # Remove the name titles
      'country': get_country(assignee), # Get the assignee country
      'legal-entity-text': assignee.get('legal-entity-text') or 'NA'
    }
    # Append assignee to assignees_info list
    assignees_info.append(assignee_info)

  # Populate transformed entry fields
  transformed_entry['assignors-info'] = assignors_info
  transformed_entry['assignees-info'] = assignees_info
  transformed_entry['property-count'] = str(len(entry['assignment']['properties']))

  # Assign transformed_entry to the rf_id key in transformed_data
  transformed_data[rf_id] = transformed_entry

# Print the first five elements of the transformed data
for i, (rf_id, entry) in enumerate(list(transformed_data.items())[:5], start=1):
    print(f"\n")
    for key, value in entry.items():
        print(f"  {key}: {value}")



  last-update-date: 2018-08-13
  conveyance-text: ASSIGNS THE ENTIRE INTEREST
  correspondent-party: WORLD HOLDINGS INC.
  assignors-info: [{'party-name': 'ISODIOL INTERNATIONAL INC', 'date-acknowledged': 'NA', 'execution-date': '2018-07-02', 'country': 'CANADA', 'legal-entity-text': 'CORPORATION'}]
  assignees-info: [{'party-name': 'WORLD HOLDINGS, INC.', 'country': 'USA', 'legal-entity-text': 'CORPORATION'}]
  property-count: 1


  last-update-date: 2008-06-09
  conveyance-text: ASSIGNS THE ENTIRE INTEREST
  correspondent-party: CHRISTIE, PARKER & HALE, LLP
  assignors-info: [{'party-name': 'EARTH ISLAND', 'date-acknowledged': 'NA', 'execution-date': '2008-05-06', 'country': 'USA', 'legal-entity-text': 'CORPORATION'}]
  assignees-info: [{'party-name': 'OLD FRIENDS HOLDINGS, LLC', 'country': 'USA', 'legal-entity-text': 'LIMITED LIABILITY COMPANY'}]
  property-count: 1


  last-update-date: 1992-08-28
  conveyance-text: CHANGE OF NAME 19820514
  correspondent-party: KEGAN, KEGAN, ET 

-------------------------------------

<div class="alert alert-block alert-warning">

## 5.  Writing to CSV/JSON File <a class="anchor" name="write"></a>

</div>

In this step, we write the processed data from the previous step, which has already been stored in a JSON format, to JSON file.

In [6]:
# Write the transformed_data to a JSON file using json.dump
with open('task1_005.json', 'w', encoding = 'utf-8') as json_file:
    json.dump(transformed_data, json_file, indent = 4) # Specify an indent of 4 for readability

-------------------------------------

<div class="alert alert-block alert-info">
    
### 5.1. Verification - using the sample files <a class="anchor" name="test_xml"></a>

Finally, we test the output JSON file using the test script provided on Google Drive to ensure the output file passes all the checks and does not raise any errors.   

In [7]:
import json
import pandas as pd
groupnum=input("Please input your group number:")
with open("task1_{}.json".format(groupnum.zfill(3)),"r") as file:
    content=file.read()
    try:
        data=json.loads(content)
    except:
        raise TypeError("Invalid json: unable to load data!")
    try:
        df=pd.DataFrame(data)
        table = pd.pivot_table(df, values=df.columns, columns=df.index, aggfunc="max")
    except Exception as e:
        raise ValueError("Invalid daya structure: check your data structure!")
    allowedvariance={"assignors_info":"assignors-info","assignees_info":"assignees-info",
                     "last-updated-date":'last-updated-date',"correspondent_party":"correspondent-party"}
    studentcols=[]
    for each in df.index.to_list():
        if each in allowedvariance.keys():
            studentcols.append(allowedvariance[each])
        else:
            studentcols.append(each)

    indexnames=['assignees-info', 'assignors-info', 'conveyance-text',
       'correspondent-party', 'last-update-date', 'property-count']
    assert set(indexnames)==set(studentcols), "Invalid key in json: check your json keys!"
    print("Task 1 json file passed!")

Please input your group number:005
Task 1 json file passed!


-------------------------------------

<div class="alert alert-block alert-warning">

## 6. Summary <a class="anchor" name="summary"></a>

</div>

In this assignment, we demonstrated the extraction, parsing, transformation, and validation of semi-structured data from trademark assignments records. The primary objective was to convert raw text data stored in XML format into a structured JSON format which is suitable for further analysis and processing. The initial phase included examining the raw data to identify patterns and data elements. The data was parsed from XML format to extract details regarding the assignments and transformed into a structured JSON format using regular expressions (regex) and custom functions for specific tasks such as removing titles from names and mapping locations to countries. Once the structured data was written to a JSON file, a test script was implemented to check key indices and validate the JSON structure against expectations, ensuring the correctness of the output file.

-------------------------------------

<div class="alert alert-block alert-warning">

## 7. References <a class="anchor" name="Ref"></a>

</div>



[1]Python Regex, https://www.geeksforgeeks.org/python-regex/.

....


## --------------------------------------------------------------------------------------------------------------------------